# Quick Test of ReLU Count
This is just a quick test to ensure ReLUCount is working properly. It uses Pytorch's Quick Start example: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import relu

In [19]:
relu.restart_count()

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [25]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            relu.ReLUCount(),
            nn.Linear(512, 512),
            relu.ReLUCount(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLUCount()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLUCount()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [26]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [27]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [28]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [29]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307807  [   64/60000]
loss: 2.290855  [ 6464/60000]
loss: 2.268301  [12864/60000]
loss: 2.258748  [19264/60000]
loss: 2.253629  [25664/60000]
loss: 2.227376  [32064/60000]
loss: 2.223784  [38464/60000]
loss: 2.188302  [44864/60000]
loss: 2.183179  [51264/60000]
loss: 2.156481  [57664/60000]
Test Error: 
 Accuracy: 54.6%, Avg loss: 2.149040 

Epoch 2
-------------------------------
loss: 2.158005  [   64/60000]
loss: 2.145666  [ 6464/60000]
loss: 2.087734  [12864/60000]
loss: 2.103222  [19264/60000]
loss: 2.053145  [25664/60000]
loss: 1.999209  [32064/60000]
loss: 2.010675  [38464/60000]
loss: 1.930114  [44864/60000]
loss: 1.937462  [51264/60000]
loss: 1.866727  [57664/60000]
Test Error: 
 Accuracy: 60.4%, Avg loss: 1.862620 

Epoch 3
-------------------------------
loss: 1.897246  [   64/60000]
loss: 1.863481  [ 6464/60000]
loss: 1.744765  [12864/60000]
loss: 1.786103  [19264/60000]
loss: 1.669147  [25664/60000]
loss: 1.631394  [32064/600

In [34]:
relu.get_relu_proportion()

6.510416666666667e-09

In [35]:
relu.ZERO_RELU_CALL

4